In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git -b get_by_ee
print( 'Done!' )

In [ ]:
%run ECHO_modules/DataSet.py
%run ECHO_modules/make_data_sets.py

### 1. Read the local spreadsheet with the carceral facilities

In [ ]:
carc_data  = pd.read_csv( "reformatted_ca_validated.csv", encoding='iso-8859-1',
                 dtype={"Total_FRS_IDS": "Int64"} )
registry_ids = carc_data['Total_FRS_IDS'].dropna().unique()
print( "Number of records in CSV = {}, number of non-NA ids = {}".format( len(carc_data), len(registry_ids)))



### 2. Look up the facilities in ECHO_EXPORTER

In [ ]:
id_list = []
for id in registry_ids:
    id_list.append( id )
    
ds = DataSet( name="exporter", base_table="ECHO_EXPORTER",
            table_name="ECHO_EXPORTER", echo_type=None, idx_field="REGISTRY_ID" )
echo_data = ds.get_data_by_ee_ids( id_list )

echo_data.to_csv( "ECHO_EXPORTER-CA_Carceral.csv")

### 3. Look up the facilities in the program data sets

In [ ]:
data_sets = make_data_sets()
for pgm_name,ds in data_sets.items():
    if ( ds.echo_type == 'SDWA' ):
        echo_flag = 'SDWIS_FLAG'
    else:
        echo_flag = ds.echo_type + '_FLAG'
    r_ids = echo_data[ echo_data[ echo_flag ] == 'Y' ].index
    print( pgm_name )
    program_data = ds.get_data_by_ee_ids( r_ids )
    if ( program_data is not None):
        program_data.to_csv( pgm_name+'.csv' )

### 4. Use this section of cells to see if a single facility is in the ECHO_EXPORTER table.  Search for it by its REGISTRY_ID

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout

id_widget = widgets.IntText(description='ID to find:', disabled=False)
display( id_widget )

In [ ]:
sql = 'select * from "ECHO_EXPORTER" where "REGISTRY_ID"=\'{}\''.format( str( id_widget.value ))
echo_data = get_data( sql )
echo_data

### 5. If the facility is in ECHO_EXPORTER, it should be found in the CSV files generated in step 3.